In [0]:
%sql
create schema if not exists inflacao.silver;

In [0]:
%sql


CREATE OR REPLACE TABLE inflacao.silver.silver_poupanca_clean AS
SELECT 
    to_date(data, 'dd/MM/yyyy') AS data_referencia,
    CAST(valor AS DOUBLE) / 100 AS taxa_mensal_decimal,
    ingestion_date
FROM inflacao.bronze.bronze_poupanca_taxas;

CREATE OR REPLACE TABLE inflacao.silver.silver_assets_brl AS
WITH cotacao_dolar AS (
    SELECT 
        to_date(date) AS data_join, 
        close AS taxa_dolar
    FROM inflacao.bronze.bronze_usd_brl_rate
),
bitcoin_brl AS (
    SELECT 
        to_date(b.date) AS data_referencia,
        'Bitcoin' AS ativo,
        (b.close * d.taxa_dolar) AS preco_brl
    FROM inflacao.bronze.bronze_bitcoin_usd b
    INNER JOIN cotacao_dolar d ON to_date(b.date) = d.data_join
),
sp500_brl AS (
    SELECT 
        to_date(s.date) AS data_referencia,
        'S&P 500' AS ativo,
        (s.close * d.taxa_dolar) AS preco_brl
    FROM inflacao.bronze.bronze_sp500_usd s
    INNER JOIN cotacao_dolar d ON to_date(s.date) = d.data_join
)

SELECT * FROM bitcoin_brl
UNION ALL
SELECT * FROM sp500_brl;